### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [1]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

### Load data and model

In [2]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Repo card metadata block was not found. Setting CardData to empty.




Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [3]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [4]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [5]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [6]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [7]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [8]:
from tqdm import tqdm

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model = model.to(device)

cuda


In [10]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [11]:
from sklearn.metrics import accuracy_score

val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=16, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=5
)
preds = []
trues = []
for batch in tqdm(val_loader):
    batch_on_dev = {k: v.to(device) for k, v in batch.items()}
    pred = model(
        input_ids=batch_on_dev['input_ids'],
        attention_mask=batch_on_dev['attention_mask'],
        token_type_ids=batch_on_dev['token_type_ids']
    )
    preds_on_dev = torch.argmax(pred.logits, dim=-1).data
    trues_on_dev = batch_on_dev['labels'].data
    preds.extend(preds_on_dev.cpu().numpy())
    trues.extend(trues_on_dev.cpu().numpy())

accuracy = accuracy_score(trues, preds)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 2527/2527 [05:02<00:00,  8.35it/s]


In [12]:
assert 0.9 < accuracy < 0.91
print(accuracy)

0.9083848627256987


### Task 2: train the model (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [13]:
%pip install transformers[sentencepiece]

In [14]:
model_name = 'microsoft/deberta-v3-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [16]:
model = model.to(device)

In [17]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   2306 MiB |   4353 MiB |   5575 GiB |   5573 GiB |
|       from large pool |   2305 MiB |   4351 MiB |   5390 GiB |   5388 GiB |
|       from small pool |      1 MiB |      2 MiB |    184 GiB |    184 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   2306 MiB |   4353 MiB |   5575 GiB |   5573 GiB |
|       from large pool |   2305 MiB |   4351 MiB |   5390 GiB |

In [23]:
train_set = qqp_preprocessed['train']
val_set = qqp_preprocessed['validation']
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=20, shuffle=True, collate_fn=transformers.default_data_collator, num_workers=4
)
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=8, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=4
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [24]:
def train_epoch(train_loader, model, loss_fn, optimizer):
  losses = []
  model.train()
  for i, batch in enumerate(tqdm(train_loader)):
      batch_on_dev = {k: v.to(device) for k, v in batch.items()}
      pred = model(
          input_ids=batch_on_dev['input_ids'],
          attention_mask=batch_on_dev['attention_mask'],
          token_type_ids=batch_on_dev['token_type_ids']
      )

      loss = loss_fn(pred.logits, batch_on_dev['labels'])
      loss.backward()
      optimizer.zero_grad()
      if i % 500 == 0:
          print(f'Curr loss: {loss.item()}')
      losses.append(loss.item())
  return losses

In [25]:
from sklearn.metrics import accuracy_score

def val_epoch(val_loader, model):
    preds = []
    trues = []
    for batch in tqdm(val_loader):
        batch_on_dev = {k: v.to(device) for k, v in batch.items()}
        pred = model(
            input_ids=batch_on_dev['input_ids'],
            attention_mask=batch_on_dev['attention_mask'],
            token_type_ids=batch_on_dev['token_type_ids']
        )
        preds_on_dev = torch.argmax(pred.logits, dim=-1).data
        trues_on_dev = batch_on_dev['labels'].data
        preds.extend(preds_on_dev.cpu().numpy())
        trues.extend(trues_on_dev.cpu().numpy())
    accuracy = accuracy_score(trues, preds)
    return accuracy

In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
import numpy as np

NUM_EPOCHS = 2

losses = []
acc = []

for epoch in range(NUM_EPOCHS):
    print(f'Started Epoch no.{epoch}')
    epoch_losses = train_epoch(train_loader, model, loss_fn, optimizer)
    losses.extend(epoch_losses)

    epoch_acc = val_epoch(val_loader, model)
    acc.append(epoch_acc)
    print(f'Epoch no.{epoch} finished. Loss: {np.mean(epoch_losses)}, Accuracy: {epoch_acc}')

Started Epoch no.0


  0%|          | 1/18193 [00:02<12:28:37,  2.47s/it]

Curr loss: 0.6827170252799988


  3%|▎         | 501/18193 [04:04<2:23:22,  2.06it/s]

Curr loss: 0.6939945220947266


  6%|▌         | 1001/18193 [08:04<2:17:57,  2.08it/s]

Curr loss: 0.7064210176467896


  8%|▊         | 1501/18193 [12:03<2:14:05,  2.07it/s]

Curr loss: 0.6933838129043579


 10%|▉         | 1766/18193 [14:11<2:10:55,  2.09it/s]

### **Закончились лимиты в Google Colab :(**

Запустил обучение, пробовал разные размеры батча, но к сожалению бесплатные ресурсы GPU закончились, а также закончилось время до дедлайна. Сдам эту версию и уже после дедлайна попробую поставить обучение снова, думаю с точки зрения идеи обучение адекватное, просто надо больше времени закладывать на этот процесс в следующий раз

### Task 3: try the full pipeline (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.